In [1]:
# requests 패키지 가져오기
import requests

# 가져올 url 문자열로 입력
url = 'https://s.search.naver.com/p/newssearch/search.naver?'

In [8]:
from urllib.parse import urlparse, parse_qs, parse_qsl, urlencode, urlunparse
from datetime import datetime

params = {
    'field': ['0'],
    'is_dts': ['0'],
    'is_sug_officeid': ['0'],
    'mynews': ['0'],
    'nqx_theme': ['{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'],
    'nso': ['&nso=so:dd,p:1h,a:all', 'so:dd,p:1h,a:all'],
    'office_category': ['0'],
    'office_section_code': ['0'],
    'office_type': ['0'],
    'photo': ['0'],
    'query': ['삼성전자'],
    'service_area': ['0'],
    'sort': ['1'],
    'spq': ['0'],
    'start': ['11'],
    'where': ['news_tab_api']
}

params['ds'] = ['2024.05.20.00.00']
params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
params['sort'] = "1"
params['start'] = "1"

parts = urlparse(url)
parts = parts._replace(query=urlencode(params, doseq=True))
new_url = urlunparse(parts)

In [6]:
from bs4 import BeautifulSoup as bs
import re

headers = {
    # "User-Agent": generate_user_agent(os='win',device_type='desktop'),
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

params['ds'] = ['2024.05.20.17.00']
params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
params['sort'] = "1"
params['start'] = "1"
parts = urlparse(url)
parts = parts._replace(query=urlencode(params, doseq=True))
new_url = urlunparse(parts)
print(new_url)

total = []

while True:
    response = requests.get(new_url)
    soup = bs(response.content, 'html.parser')
    naver_news_links = [a['href'] for a in soup.find_all('a', string='네이버뉴스') if a['href']]

    for link in naver_news_links:
        tmp = {}
        article_res = requests.get(link[2:-2])
        article_soup = bs(article_res.content, 'html.parser')
        title = article_soup.select_one('#title_area > span').get_text(strip=True)
        tmp['title'] = title
        time = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')['data-date-time']
        tmp['date_news'] = time
        try:
            summary = article_soup.select_one('#dic_area > strong').get_text(strip=True)
        except AttributeError:
            summary = None
        tmp['summary'] = summary
        for tag in article_soup.find_all(class_='end_photo_org'):
            tag.decompose()
        main = article_soup.select_one('#dic_area').get_text(strip=True).replace("\n\n\n", "")
        tmp['content'] = main
        tmp['date_collect'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        tmp['url'] = link[2:-2]

        total.append(tmp)
        if tmp in total:
            break
    params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
    params['start'] = str(int(params['start'])+10)
    parts = parts._replace(query=urlencode(params, doseq=True))
    new_url = urlunparse(parts)

https://s.search.naver.com/p/newssearch/search.naver?field=0&is_dts=0&is_sug_officeid=0&mynews=0&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Add%2Cp%3A1h%2Ca%3Aall&nso=so%3Add%2Cp%3A1h%2Ca%3Aall&office_category=0&office_section_code=0&office_type=0&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&service_area=0&sort=1&spq=0&start=1&where=news_tab_api&ds=2024.05.20.17.00&de=2024.05.20.18.17


KeyboardInterrupt: 

In [12]:
# requests 패키지 가져오기
import requests
from urllib.parse import urlparse, urlencode, urlunparse
from datetime import datetime
from bs4 import BeautifulSoup as bs

# 기본 URL 설정
url = 'https://s.search.naver.com/p/newssearch/search.naver?'

# 초기 파라미터 설정
params = {
    'field': ['0'],
    'is_dts': ['0'],
    'is_sug_officeid': ['0'],
    'mynews': ['0'],
    'nqx_theme': ['{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'],
    'office_category': ['0'],
    'office_section_code': ['0'],
    'office_type': ['0'],
    'photo': ['0'],
    'query': ['삼성전자'],
    'service_area': ['0'],
    'sort': '1',
    'start': '1',
    'where': ['news_tab_api'],
    'ds': ['2024.05.20.00.00'],
    'de': [datetime.now().strftime("%Y.%m.%d.%H.%M")]
}

# URL 생성 함수
def make_url(base_url, params):
    parts = urlparse(base_url)
    parts = parts._replace(query=urlencode(params, doseq=True))
    return urlunparse(parts)



# 수집 데이터 리스트 초기화
total = []

while True:
    new_url = make_url(url, params)
    response = requests.get(new_url, headers=headers)
    soup = bs(response.content, 'html.parser')
    naver_news_links = [a['href'] for a in soup.find_all('a', string='네이버뉴스') if a['href']]

    if not naver_news_links:  # 새로운 데이터가 없을 경우 반복 중단
        break

    for link in naver_news_links:
        tmp = {}
        article_res = requests.get(link[2:-2])
        article_soup = bs(article_res.content, 'html.parser')
        title = article_soup.select_one('#title_area > span').get_text(strip=True)
        tmp['title'] = title
        time = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')['data-date-time']
        tmp['date_news'] = time
        try:
            summary = article_soup.select_one('#dic_area > strong').get_text(strip=True)
        except AttributeError:
            summary = None
        tmp['summary'] = summary
        for tag in article_soup.find_all(class_='end_photo_org'):
            tag.decompose()
        main = article_soup.select_one('#dic_area').get_text(strip=True).replace("\n\n\n", "")
        tmp['content'] = main
        tmp['date_collect'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        tmp['url'] = link[2:-2]

        total.append(tmp)

    params['start'] = str(int(params['start']) + 10)

print(len(total))

106


In [18]:
a = "https://s.search.naver.com/p/newssearch/search.naver?de=2024.05.20.19.34&ds=2024.05.20.18.34&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=0&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Add%2Cp%3A1h%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=7&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&query_original=&service_area=1&sort=1&spq=0&start=21&where=news_tab_api&nso=so:dd,p:1h,a:all&_callback=jQuery112407536005779775927_1716201250363&_=1716201250365"

parse_qsl(urlparse(a).query)

[('de', '2024.05.20.19.34'),
 ('ds', '2024.05.20.18.34'),
 ('field', '0'),
 ('is_dts', '0'),
 ('is_sug_officeid', '0'),
 ('mynews', '0'),
 ('nqx_theme',
  '{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'),
 ('nso', '&nso=so:dd,p:1h,a:all'),
 ('office_category', '0'),
 ('office_section_code', '0'),
 ('office_type', '0'),
 ('pd', '7'),
 ('photo', '0'),
 ('query', '삼성전자'),
 ('service_area', '1'),
 ('sort', '1'),
 ('spq', '0'),
 ('start', '21'),
 ('where', 'news_tab_api'),
 ('nso', 'so:dd,p:1h,a:all'),
 ('_callback', 'jQuery112407536005779775927_1716201250363'),
 ('_', '1716201250365')]

In [19]:
a = "https://s.search.naver.com/p/newssearch/search.naver?de=2024.05.20.19.45&ds=2024.05.20.17.45&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=0&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Add%2Cp%3Aall%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=8&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&query_original=&service_area=0&sort=1&spq=0&start=11&where=news_tab_api&nso=so:dd,p:all,a:all&_callback=jQuery112404406811477406485_1716201920061&_=1716201920062"
parse_qsl(urlparse(a).query)

[('de', '2024.05.20.19.45'),
 ('ds', '2024.05.20.17.45'),
 ('field', '0'),
 ('is_dts', '0'),
 ('is_sug_officeid', '0'),
 ('mynews', '0'),
 ('nqx_theme',
  '{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'),
 ('nso', '&nso=so:dd,p:all,a:all'),
 ('office_category', '0'),
 ('office_section_code', '0'),
 ('office_type', '0'),
 ('pd', '8'),
 ('photo', '0'),
 ('query', '삼성전자'),
 ('service_area', '0'),
 ('sort', '1'),
 ('spq', '0'),
 ('start', '11'),
 ('where', 'news_tab_api'),
 ('nso', 'so:dd,p:all,a:all'),
 ('_callback', 'jQuery112404406811477406485_1716201920061'),
 ('_', '1716201920062')]

In [20]:
a = "https://s.search.naver.com/p/newssearch/search.naver?de=&ds=&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=0&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Add%2Cp%3Aall%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=0&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&query_original=&service_area=0&sort=1&spq=0&start=11&where=news_tab_api&nso=so:dd,p:all,a:all&_callback=jQuery112405132723700993316_1716202227285&_=1716202227286"

parse_qsl(urlparse(a).query)

[('field', '0'),
 ('is_dts', '0'),
 ('is_sug_officeid', '0'),
 ('mynews', '0'),
 ('nqx_theme',
  '{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'),
 ('nso', '&nso=so:dd,p:all,a:all'),
 ('office_category', '0'),
 ('office_section_code', '0'),
 ('office_type', '0'),
 ('pd', '0'),
 ('photo', '0'),
 ('query', '삼성전자'),
 ('service_area', '0'),
 ('sort', '1'),
 ('spq', '0'),
 ('start', '11'),
 ('where', 'news_tab_api'),
 ('nso', 'so:dd,p:all,a:all'),
 ('_callback', 'jQuery112405132723700993316_1716202227285'),
 ('_', '1716202227286')]